First model to classify titanic classifier

### Reading data from file

In [1]:
### Lendo os dados e separando o label
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('input/train.csv', index_col='PassengerId') 
X_test = pd.read_csv('input/test.csv', index_col='PassengerId')
X = X_full.copy()

X.drop(['Cabin','Name','Ticket'], axis=1, inplace=True)
X.drop( X[ X['Embarked'].isnull() ].index, axis=0, inplace=True)

In [2]:
# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['Survived'], inplace=True)
y = X.Survived
X.drop(['Survived'], axis=1, inplace=True)

In [ ]:
# X.shape

In [ ]:
# y.shape

### Inspecting data to get some insight

In [ ]:
# X.head()

In [ ]:
# X.tail()

In [ ]:
# X.shape

In [ ]:
# y.shape

In [ ]:
# X.describe()

In [ ]:
# X.info()

In [ ]:
# X['Embarked'].unique()

In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# X.hist(bins=25,figsize=(20,15)) 
# plt.show()

In [ ]:
# corr_matrix = X_full.corr()
# corr_matrix['Survived'].sort_values(ascending=False)

In [ ]:
# from pandas.plotting import scatter_matrix

# attributes = ['Fare','Parch', 'Pclass'] #

# scatter_matrix(X_full[attributes], figsize=(20,15))
# plt.show()

In [ ]:
# X_full.plot(kind='scatter', x='Fare', y='Survived', alpha=0.1)
# plt.show()

### So far I have not gained any insights from the data. I'll get rid of the Cabin feature which has tons of missing data and I'll imput the missing data in Age and Embarked features

In [ ]:
# Nothing that worked. I've done part of the missing variable treatment above

# X.Embarked.isnull() 
# pd.isnull(X['Embarked'])
# X['Embarked'].unique()
# X['Embarked'].value_counts()
# X.drop()
# X.index
# X.drop(X['Embarked'].isnull(),axis=1,inplace=True)

# print(X_full[X_full['Embarked'].isnull()])
# 62 e 830
# X.loc[[62,830]] # por label
# X.iloc[829] # por índice
# X_drop = X.drop(X.loc[[62,830]], axis=0)
# X.drop(['Cabin'], axis=1, inplace=True)
# X.drop([62,830], axis=0, inplace=True)
# y.drop([62,830], axis=0, inplace=True)


# Maybe I should drop lines before dropping y, above.

# X.drop(['Cabin'], axis=1, inplace=True)
# X.drop( X[ X['Embarked'].isnull() ].index, axis=0, inplace=True)
# y.drop([62,830], axis=0, inplace=True)

In [ ]:
# X.shape

In [ ]:
# X.head()

### General transformation in data

In [3]:
X['Pclass'] = X['Pclass'].astype('category')
X_test['Pclass'] = X_test['Pclass'].astype('category')

In [ ]:
# X.info()

In [4]:
# X[ X['Parch'].isnull() ].index
# X['Parch'].isnull() 
# X['Ratio_childern_parents'] = X['SibSp'] / X['Parch']
# Z = X['SibSp'] / X['Parch']
# pd.isnull(X['Parch'])
# X['Parch'].unique()
# X['SibSp'].unique()
# X[ (X['Parch']==0) & (X['SibSp'] == 0)]
# X[ (X['SibSp'] ==0) ]
# X[ X['Parch'] == 0 ]
# X['Age'].unique()


def alone(row):
    if row['SibSp'] == 0 and row['Parch'] == 0:
        val = 1
    else:
        val = 0
    return val

def noParents(row):
    if row['Parch'] == 0:
        val = 1
    else:
        val = 0
    return val

def childPerParents(row):
    if row['Parch'] == 0:
        val = 0
    else:
        val = row['SibSp']/row['Parch']
    return val
        
# New column for people traveling alone
X['Alone'] = X.apply(alone, axis=1)
X_test['Alone'] = X_test.apply(alone, axis=1)

# New column for parents/no parents
X['NoParents'] = X.apply(noParents, axis=1)
X_test['NoParents'] = X_test.apply(noParents, axis=1)

X['ChildPerParent'] = X.apply(childPerParents, axis=1)
X_test['ChildPerParent'] = X_test.apply(childPerParents, axis=1)

In [ ]:
# X.tail()
# X['Age'].unique()
# X.tail()
# X.shape

### Splitting the training data

In [5]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [ ]:
# y_train.shape

In [6]:
### Achando as colunas categóricas e numéricas
categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 10 and 
                    X_train[cname].dtype == "object"]


# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
# my_cols = categorical_cols + numerical_cols
# X_train = X_train[my_cols].copy()
# X_valid = X_valid[my_cols].copy()
# X_test = X_test[my_cols].copy()

In [ ]:
# Data columns (total 7 columns):
#  #   Column    Non-Null Count  Dtype  
# ---  ------    --------------  -----  
#  0   Pclass    889 non-null    int64  
#  1   Sex       889 non-null    object 
#  2   Age       712 non-null    float64
#  3   SibSp     889 non-null    int64  
#  4   Parch     889 non-null    int64  
#  5   Fare      889 non-null    float64
#  6   Embarked  889 non-null    object 

In [ ]:
# categorical_cols

In [ ]:
# numerical_cols

### Imputing missing data

#### Numerical

In [7]:
# Make copy to avoid changing original data (when imputing)
num_X_train = X_train[numerical_cols].copy()
num_X_valid = X_valid[numerical_cols].copy()
num_X_test = X_test[numerical_cols].copy()

from sklearn.impute import SimpleImputer
numerical_transformer = SimpleImputer(strategy='median')

num_X_train = pd.DataFrame(numerical_transformer.fit_transform(num_X_train), columns = num_X_train.columns)
num_X_valid = pd.DataFrame(numerical_transformer.transform(num_X_valid), columns = num_X_valid.columns)
num_X_test = pd.DataFrame(numerical_transformer.transform(num_X_test), columns = num_X_test.columns)

num_X_train.index = X_train.index
num_X_valid.index = X_valid.index
num_X_test.index = X_test.index

In [ ]:
# num_X_train['Age'].unique()
# y_train.shape

In [8]:
# z = list(X['Age'])
# print(z)
# X.loc[[889] , ['Age']]
# z = X.at[888,'Age']
# int(str(z)[-1:]) == 5
# int(str(row['Age'])[-1:]) == 5
import numpy as np

def isAprox(row):
    if int(str(row['Age'])[-1:]) == 5:
        val = 1
    else:
        val = 0
    return val

num_X_train['IsAprox'] = num_X_train.apply(isAprox, axis=1)
num_X_valid['IsAprox'] = num_X_valid.apply(isAprox, axis=1)
num_X_test['IsAprox'] = num_X_test.apply(isAprox, axis=1)

# X_test['IsAprox'] = X_test.apply(isAprox, axis=1)
# X.tail()

In [ ]:
# num_X_train['IsAprox'].unique()

In [ ]:
# num_X_train.info()

In [ ]:
# num_X_train.head()

In [ ]:
# num_X_valid.info()

In [ ]:
# num_X_train.shape
# y_train.shape

In [ ]:
# cat_X_train = X_train[categorical_cols].copy()
# cat_X_valid = X_valid[categorical_cols].copy()
# cat_X_test = X_test[categorical_cols].copy()

In [ ]:
# cat_X_train.info()

In [ ]:
# cat_X_train.head()

In [ ]:
# cat_X_train.shape

### Categorical Encoding

In [9]:
cat_X_train = X_train[categorical_cols].copy()
cat_X_valid = X_valid[categorical_cols].copy()
cat_X_test = X_test[categorical_cols].copy()


from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_X_train = pd.DataFrame(OH_encoder.fit_transform(cat_X_train))
OH_X_valid = pd.DataFrame(OH_encoder.transform(cat_X_valid))
OH_X_test = pd.DataFrame(OH_encoder.transform(cat_X_test))

# One-hot encoding removed index; put it back
OH_X_train.index = cat_X_train.index
OH_X_valid.index = cat_X_valid.index
OH_X_test.index = cat_X_test.index

In [ ]:
# print(OH_X_train.shape)
# print(num_X_train.shape)

In [ ]:
# OH_X_train.info()
# OH_X_train.head()
# num_X_train.head()

In [ ]:
# OH_X_train.head()

In [10]:
all_X_train = pd.concat([num_X_train, OH_X_train], axis=1)
all_X_valid = pd.concat([num_X_valid, OH_X_valid], axis=1)
all_X_test = pd.concat([num_X_test, OH_X_test], axis=1)
# all_X_test.head()

In [ ]:
# all_X_train.head()

In [ ]:
# all_X_train.tail()

In [ ]:
# all_X_train.shape

In [ ]:
# new_df = pd.concat([all_X_train, y_train], axis=1)
# new_df.head()

In [ ]:
# corr_matrix = new_df.corr()
# corr_matrix['Survived'].sort_values(ascending=False)

### Defining the model

In [11]:
# ### Definindo o modelo
from xgboost import XGBClassifier

# # Define the model
model = XGBClassifier(objective ='binary:logistic', n_estimators=1000, learning_rate=0.05) 

In [ ]:
# all_X_train.shape

In [ ]:
# y_train.shape

### Training the model

In [12]:
model.fit(all_X_train, y_train)

# preds_test = model.predict(OH_imputed_X_test)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

### Evaluating

In [13]:
from sklearn.metrics import mean_absolute_error

predicted_survival = model.predict(all_X_valid)
mean_absolute_error(y_valid, predicted_survival)

0.23595505617977527

### Generating predictions

In [14]:
# # Preprocessing of test data, fit model
preds_test = model.predict(all_X_test) 

output = pd.DataFrame({'PassengerId': all_X_test.index, 'Survived': preds_test})
output.to_csv('output/submission.csv', index=False)

### Next Steps

* Cabin number and Ticket may have more information, depending on how they built those features. It needs further investigation.